<h1 style="display:inline-block">Iot Essay</h1><h2>Sample Python code to implement IoT</h2>

This is a no-longer-mantained code to add an abstraction layer to vpython. This code used to be hosted in a cloud function on GCP, and a jupyter NB will call it add 3D capabilities to the cells.

In [ ]:
def send_code( request ):
    return """

# TODO: implementar box

import math
import numpy as np
from io import StringIO
import re

model = { "asset": 
    {
        "id": "clip-bolt",
        "unit": 
        {
            "name": "milimeter",
            "meter": 0.001
        },

        # length of curve segments for development
        "curve-precision-dev": 0.25,

        # length of curve segments for final production model
        "curve-precision-prod": 0.025,
    },

    # model is his own parent can be a strange thing to do but this is used 
    # in the algorithm that checks if arguments passed as targets are in the
    # model tree
    # TODO: os targets ainda nao estao implementados e nem sei qual a verdadeira utilidade
    "parent": None
}


model[ "parent" ] = model


def check_tree( item, id, tgt = None ):
    global model

    if tgt is None:
        tgt = model

    # assert target is in model tree
    tree = tgt
    while tree[ "parent" ] != model:
        tree = tree[ "parent" ]

    if item not in tgt:
        tgt[ item ] = {}

    if id not in tgt[ item ]:
        tgt[ item ][ id ] = None

    return tgt


def assert_model( path_array ):
    ok = True

    try:
        node = model[ path_array[ 0 ]]
    except:
        ok = False

    if ok:
        try:
            for i in range( 1, len( path_array )):
                el = node[ path_array[ i ]]
        except:
            ok = False

    return ok


# sketches will be always 2D and on xy plane
class Sketch:
    def __init__( self ):
        pass

    @staticmethod
        # TODO: target will allow to position a sketch in any position in the model tree
        # I don´t know yet if this is useful. maybe I should force that sketches
        # will be always directly in the tree root
    def add( id, geom, target = None ):
        tgt = check_tree( "sketch", id, target )

        tgt[ "sketch" ][ id ] = { "id": id, "geom": geom }


class Curve:
    def __init__( self ):
        pass

    @staticmethod
    def circle( radius, pos = ( 0., 0., 0. )):
        return { "category": "curve", "type": "circle", "radius": radius, "pos": pos }

class Surface:
    def __init__( self ):
        pass

    @staticmethod
    def move( surface, direction, length ):
        ndir = direction / np.linalg.norm( direction )
        ndir = ndir * length

        nface = surface.copy()

        for i in range( len( nface[ "v" ])):
            nface[ "v" ][ i ] = tuple( nface[ "v" ][ i ] + ndir )

        return nface


    @staticmethod
    def precision():
        _prec = "curve-precision-"
        _prec += "dev" if Implement.mode.development else "prod"

        if assert_model([ "asset", _prec ]):
            prec = float( model[ "asset" ][ _prec ])
        else:
            prec = 0.25 if Implement.mode.development else 0.025

        return prec


    @staticmethod
    def face( sketch ):
        if "sketch" not in model:
            s_err = "ERROR: there is no sketches on the model but the sketch " + sketch + " is referenced." + chr( 10 )
            # TODO: trocar esses prints - enviar os erros para dentro da tecnologia de saida
            return { "print": s_err }
        else:
            if sketch not in model[ "sketch" ]:
                s_err = "ERROR: referenced sketch " + sketch + " is not found." + chr( 10 )
                return { "print": s_err }

        res = {}

        # identify the method used to create the sketch
        sk = model[ "sketch" ][ sketch ]

        # up vector of all sketches is 0,0,1
        up = (0.,0.,1.)
        #up = np.array([ up[ 0 ], up[ 1 ], up[ 2 ]])
        #up /= np.linalg.norm( up )

        # dev or prod?
        prec = Surface.precision()

        geom = sk[ "geom" ]
        _geom = geom[ "type" ]

        _e = _exporter

        # some capabilities of some output technologies don't need faces.
        _vns = True
        grammar = _e.grammar[ _e.syntax ]
        if "capabilities" in grammar:
            if "extrusion" in grammar[ "capabilities" ]:
                grammar = _e.grammar[ _e.syntax ][ "capabilities" ][ "extrusion" ]
                if "vertex-normals" in grammar:
                    _vns = grammar[ "vertex-normals" ]

        if _geom == "circle":
            # generate vertices
            res[ "v" ] = [] # vertices

            if _vns:
                res[ "vn" ] = [] #vertex normals

            # circle perimeter
            r = float( geom[ "radius" ])
            p = 2. * math.pi * r

            # steps, angular step
            steps = p / prec;
            astp = 360. / steps

            # ct is a point in space, not vector
            ct = geom[ "pos" ]

            # two perp vects
            x, y, z = up[ 0 ], up[ 1 ], up[ 2 ]
            cx, cy, cz = ct[ 0 ], ct[ 1 ], ct[ 2 ]
            vs = []
            vs.append(( z, z, -x -y ))
            vs.append(( -y -z, x, x ))
            idx = ((z != 0) & ( -x != y )) 
            pv1 = vs[ idx ]
            pv1 = tuple( pv1 / np.linalg.norm( pv1 ))

            pv2 = np.cross( up, pv1 )
            pv2 = tuple( pv2 / np.linalg.norm( pv2 ))

            for a in range( int( steps ) + 1 ):         
                cosT = math.cos( math.radians( astp * float( a )))
                sinT = math.sin( math.radians( astp * float( a )))

                res[ "v" ].append(( cx + r * cosT * pv1[ 0 ] + r * sinT * pv2[ 0 ],
                                    cy + r * cosT * pv1[ 1 ] + r * sinT * pv2[ 1 ],
                                    cz + r * cosT * pv1[ 2 ] + r * sinT * pv2[ 2 ]))

                if _vns:
                    res[ "vn" ].append( tuple( res[ "v" ][ -1 ] / np.linalg.norm( res[ "v" ][ -1 ])))

            # close the circle
            res[ "v" ].append( res[ "v" ][ 0 ])
            if _vns:
                res[ "vn" ].append( res[ "vn" ][ 0 ])

        # edges and wire implied in circle
        return res


class Solid:
    def __init__( self ):
        pass

    @staticmethod
    def extrude( id, sketch, length, target = None ):
        tgt = check_tree( "solid", id, target )

        # check if the sketch exists
        _sketch = tgt[ "sketch" ][ sketch ]

        tgt[ "solid" ][ id ] = { "id": id, "cmd": "extrude", "sketch": sketch, "length": length }

        # TODO: por enquanto o track é gerado seguindo o "up" (0,0,1). No futuro, dar a possibilidade
        # de fornecer o track (se bem q tvz outro nome em vez de "extrusion" seja o ideal pra isso,
        # embora no vpython chame-se de extrusion com tracks qqs - ate mesmo fechados )
        tgt[ "solid" ][ id ][ "track" ] = [( 0., 0., 0. ), ( 0., 0., float( length ))]


    @staticmethod
    def instance( id, template, pos, rot=( 0., 0., 0. )):
        tgt = check_tree( "instance", id )

        # check if the instance exists
        _inst = tgt[ "instance" ][ id ]

        # check if the template exists
        _templ = tgt[ "solid" ][ template ]

        tgt[ "instance" ][ id ] = { "id": id, "template": template, "pos": pos, "rot": rot }


class _exporter:
    def __init__( self ):
        pass

    grammar = { "vpython": {
            "rem": { "open": "# ", "close": chr( 10 )},
            "line-width": 128,
            "verbose": True,  # need to generate vertices and faces
            "valid-names": {
                "rexp": "[-\\.]",
                "repl": "_"
            },
            "vertex": {
                "open": {
                    "rexp": "^(.*)$", # replace sketch name
                    "repl": r"\\1_vxs = ["
                },
                "rexp": r"\\((-?[0-9]*\\.[0-9]+),[ ]?(-?[0-9]*\\.[0-9]+),.+\\)", # replace str( vx )
                "repl": r"[ \\1, \\2 ], ", # 2D on vpython, ignoring Z (sketches are always on xy plane anyway)
                "close": {
                    "rexp": "^(.*)$", # replace sketch name
                    "repl": "]"
                },
            }, # TODO: outra forma de eliminar normais seria usar o replace pra gerar uma saida zerada
                # mas como pode ser q as normais sejam necessarias em alguns casos e outros nao, nao sei se 
                # eh boa ideia
            "vertex-normal": {
                "open": {
                    "rexp": "^(.*)$",
                    "repl": r"\\1_vns = [" 
                },
                "rexp": r"\\((-?[0-9]*\\.[0-9]+),[ ]?(-?[0-9]*\\.[0-9]+),.+\\)", # replace str( vx )
                "repl": r"[ \\1, \\2 ], ", # 2D on vpython, ignoring Z (sketches are always on xy plane anyway)
                "close": {
                    "rexp": "^(.*)$",
                    "repl": "]" 
                },
            },
            "face": {
                "rexp": "(.*)",
                "repl": "\\1 # vpython face not implemented" + chr( 10 )
            },
            "translation": {
                "code": [ 
"{# name {# instance.id #}#}.pos = vec{# instance.pos #}",
"" 
                        ],
            },
            "rotation": {
                "code": [ 
"{# name {# instance.id #}#}.rotate( angle = ( pi / 180. ) * {# instance.rot|x #}, axis = vector( 1., 0., 0. ))",
"{# name {# instance.id #}#}.rotate( angle = ( pi / 180. ) * {# instance.rot|y #}, axis = vector( 0., 1., 0. ))",
"{# name {# instance.id #}#}.rotate( angle = ( pi / 180. ) * {# instance.rot|z #}, axis = vector( 0., 0., 1. ))",
"" 
                        ],
                "params": {
                    "instance.rot|x": {
                        "rexp": r"^\\([ ]*(-?[0-9]*\\.[0-9]*),.+$",
                        "repl": r"\\1"
                    },
                    "instance.rot|y": {
                        "rexp": r"^\\(.+,[ ]*(-?[0-9]*\\.[0-9]*),.+$",
                        "repl": r"\\1"
                    },
                    "instance.rot|z": {
                        "rexp": r"^\\(.+,[ ]*(-?[0-9]*\\.[0-9]*)[ ]*\\)$",
                        "repl": r"\\1"
                    }
                }
            },
            "capabilities": {
                "extrusion": {
                    "faces": False,     # vpython don't need face data in extrusions, only vertices
                    "vertex-normals": False,
                    "code": [
"{# name {# solid.id #}#}_path = {# solid.track #}",
"{# name {# solid.id #}#} = extrusion( path={# name {# solid.id #}#}_path, shape={# name {# solid.id #}#}_vxs, color=color.gray( 0.9 ))",
"{# name {# solid.id #}#}.visible = False",
""
                            ],
                    "params": {
                        "solid.track": {
                            "rexp": r"\\(",
                            "repl": "vec("
                        }
                    }
                }
            },
            "instance": {
                "code": [
"{# name {# instance.id #}#} = compound([ {# name {# solid.id #}#}.clone()], origin = " + \
    "vec( 0., 0., {# name {# solid.id #}#}.size.z / -2.0 ))",
""
                ]
            },
            "initialize": [ 
                "from vpython import *",
                "",
                "ctr = vec( 150, 50, 150 )",
                "",
                "# This is needed in Jupyter notebook and lab to make programs easily rerunnable",
                "scene = canvas( center=ctr, background = color.gray( 0.8 ))", 
                "scene.autoscaling = False",
                "scene.forward = vec( 0, 200, 0 )",
                "scene.up = vec( 0, 0, 1 )",
                "scene.camera.pos = vec( 150, -330, 150 )",
                "scene.range = 50",
                "" ],
            "finalize": [ 
                "run = True",
                "",
                "def runner(r):",
                "    global run",
                "    run = r.checked",
                "",
                "checkbox( bind=runner, text='Run', checked=True )",
                "",
                "t, dt = 0, 0.01",
                "while True:",
                "    rate( 100 )",
                "    if run:",
                "        t += dt",
                "    else:",
                "        break",
                "" ],
        },
        "openscad": {
            "rem": { "open": "// ", "close": chr( 10 )},

            "verbose": False,       # in openscad there is no need to generate vertices and faces
            "valid-names": {
                "rexp": "[-\\.]",
                "repl": "_"
            },
            "declarative": True,   # openSCAD implements a declarative language

            # different paths in openscad have different language commands
            # other languages do not need this high level curve replacements
            "curves": {
                "circle": {
                    "code": [
"    translate( {# sketch.geom.pos #} )",                        
"        circle( {# sketch.geom.radius #}, $fn= ( 2. * PI * {# sketch.geom.radius #}) / {# precision #} );",
                            ],
                    "params": {
                        "sketch.geom.pos": {
                            "rexp": r"[\\(\\)]",
                            "repl": ""
                        }
                    }
                }
            },
            "translation": { "code": [ 
"// {# instance.id #}",
"translate({# instance.pos #})",
"    {# name {# solid.id #}#}();",
""
                                  ],
                "params": {
                    "instance.pos": {
                        "rexp": r"[\\(\\)]",
                        "repl": ""
                    }
                }
            },
            "capabilities": {
                "extrusion": {
                    "faces": False,     # vpython don't need face data in extrusions, only vertices
                    "vertex-normals": False,
                    "code": [
"module {# name {# solid.id #}#}()",
"{",
"    linear_extrude( height = {# solid.length #}, center = false, convexity = 10, twist = 0, slices = 1, $fn = 16 )",
"    {=> sketch.{# solid.sketch #}#}",
"}",
""
                            ],
                    "params": {
                        "solid.track": {
                            "rexp": r"[\\(]",
                            "repl": "vec("
                        }
                    }
                }
            },
            "initialize": "",
            "finalize": ""
        }
    }

    syntax = "vpython"


    @staticmethod
    def comment( txt ):
        grammar = _exporter.grammar[ _exporter.syntax ]
        return grammar[ "rem" ][ "open" ] + str( txt ) + grammar[ "rem" ][ "close" ]


    @staticmethod
    def verbose():
        grammar = _exporter.grammar[ _exporter.syntax ]
        return grammar[ "verbose" ]


    @staticmethod
    def initialize( strio ):
        grammar = _exporter.grammar[ _exporter.syntax ]

        if "initialize" in grammar:
            init_code = "\\n".join( grammar[ "initialize" ])

            if len( init_code.replace( " ", "" )) > 2 :
                _exporter.write( strio, init_code )


    @staticmethod
    def finalize( strio ):
        grammar = _exporter.grammar[ _exporter.syntax ]

        if "finalize" in grammar:
            end_code = "\\n".join( grammar[ "finalize" ])

            if len( end_code.replace( " ", "" )) > 2 :
                _exporter.write( strio, end_code )


    @staticmethod
    def vertex( vx ):
        vx_grammar = _exporter.grammar[ _exporter.syntax ][ "vertex" ] 
        return re.sub( vx_grammar[ "rexp" ], vx_grammar[ "repl" ],  str( vx )) 


    @staticmethod
    def vertex_normal( vn ):
        vn_grammar = _exporter.grammar[ _exporter.syntax ][ "vertex-normal" ] 
        return re.sub( vn_grammar[ "rexp" ], vn_grammar[ "repl" ],  str( vn )) 


    @staticmethod
    def face( f ):
        face_grammar = _exporter.grammar[ _exporter.syntax ][ "face" ] 
        return re.sub( face_grammar[ "rexp" ], face_grammar[ "repl" ],  str( f )) 


    _buffer = ""

    @staticmethod
    def set_buffer( txt ):
        _exporter._buffer = txt

    @staticmethod
    def get_buffer():
        return _exporter._buffer

    @staticmethod
    def write( strio, txt ):
        _this = _exporter

        grammar = _this.grammar[ _this.syntax ]

        # we don't care with number os chars per line: no special
        # procedure, _exporter.write = stringio.write
        if "line-width" not in grammar:
            strio.write( txt )
            return

        lim = int( grammar[ "line-width" ])

        # should not happen, but if buffer already bursted, print it
        # or if buffer will burst if we add txt to it, print it
        buf_len = len( _this.get_buffer()) 
        if buf_len > lim or ( buf_len + len( txt )) > lim:
            strio.write( _this.get_buffer() + chr( 10 ))
            _this.set_buffer( "" )

        # if we send an chr( 10 ), print buffer and txt
        # we already now that this will not burst the buffer
        # the only way to prevent this behavior is use flush before write
        # (or chr(10) at the begining of the txt )
        if txt[ -1 ] == chr( 10 ):
            strio.write( _this.get_buffer() + txt )
            _this.set_buffer( "" )
            return

        # in all other cases, accumulate text 
        buf_len = len( _this.get_buffer()) 
        if ( buf_len + len( txt )) < lim:
            _this.set_buffer( _this.get_buffer() + txt )
            return

        # can only get here if len( txt ) > lim
        strio.write( _this.get_buffer() + chr( 10 ))
        strio.write( txt + chr( 10 ))
        _this.set_buffer( "" )


    @staticmethod
    def flush( strio ):
        _this = _exporter

        if _this.get_buffer() != "":
            strio.write( _this.get_buffer() + chr( 10 ))
            _this.set_buffer( "" )


    @staticmethod
    def output_vertices( strio, templ_name, design ):
        _this = _exporter

        _this.write( strio, _this.comment( "vertices" ))

        grammar = _this.grammar[ _this.syntax ]
        template_id = templ_name

        if "valid-names" in grammar:
            rexp = grammar[ "valid-names" ]
            template_id = re.sub( rexp[ "rexp" ], rexp[ "repl" ], templ_name )

        grammar = _this.grammar[ _this.syntax ][ "vertex" ]

        if "open" in grammar:
            rexp = grammar[ "open" ]
            _this.write( strio, re.sub( rexp[ "rexp" ], rexp[ "repl" ], template_id ))

        for fnum in range( 0, len( design[ "faces" ])):
            # TODO: per-face open and close se precisar definir estruturas de dados
            # de vertices separadamente para cada face (nenhum formato ate agora precisa)
            face = design[ "faces" ][ fnum ]
            for vx in face[ "v" ]:
                # TODO: enviar mais dados para essa func se os vertices precisarem ser
                # identificados de forma mais especifica (por face p ex, nenhum formato
                # ate agora precisa)
                _this.write( strio, _this.vertex( vx ))

        if "close" in grammar:
            rexp = grammar[ "close" ]
            _this.write( strio, re.sub( rexp[ "rexp" ], rexp[ "repl" ], template_id ))

        _this.flush( strio )

        #TODO: mesmos TODOs acima para vertices repetem para normais por vertice

        # vertex normals, if declared / appliable
        have_vns = False

        for face in design[ "faces" ]:
            if have_vns:
                break

            if "vn" in face:
                have_vns = True
                break

        if have_vns:
            _this.write( strio, _this.comment( "vertex normals" ))

            grammar = _this.grammar[ _this.syntax ][ "vertex-normal" ]

            if "open" in grammar:
                rexp = grammar[ "open" ]
                _this.write( strio, re.sub( rexp[ "rexp" ], rexp[ "repl" ], template_id ))

            for fnum in range( 0, len( design[ "faces" ])):
                face = design[ "faces" ][ fnum ]
                for vn in face[ "vn" ]:
                    _this.write( strio, _this.vertex_normal( vn ))

            if "close" in grammar:
                rexp = grammar[ "close" ]
                _this.write( strio, re.sub( rexp[ "rexp" ], rexp[ "repl" ], template_id ))

        _this.flush( strio )


    @staticmethod
    def output_faces( strio, templ_name, design ):
        _this = _exporter

        # some capabilities of some output technologies don't need faces.
        _output = True
        grammar = _this.grammar[ _this.syntax ]
        if "capabilities" in grammar:
            if "extrusion" in grammar[ "capabilities" ]:
                grammar = _this.grammar[ _this.syntax ][ "capabilities" ][ "extrusion" ]
                if "faces" in grammar:
                    _output = grammar[ "faces" ]

        if not _output:
            return

        _exporter.write( strio, _exporter.comment( "faces" ))

        for face in template[ "faces" ]:
            # if there is no edges, a natural order is implied
            if "f" not in face:
                _f = list( range( int( face[ "vx_offset" ]), int( face[ "vx_offset" ]) + len( face[ "v" ])))
                _f.append( int( face[ "vx_offset" ]))
                _f = str( tuple( _f ))
            else:
                _f = face[ "f" ]

        _exporter.write( strio, _exporter.face( strio, solid_name, _f ))


    @staticmethod
    def _generate_code( template_name, instance_name, code_template_list, params_dict, extra_context=None, ctxt_name="" ):
        _this = _exporter

        templ = model[ "solid" ][ template_name ]

        inst = None
        if not instance_name is None and instance_name != "": 
            inst = model[ "instance" ][ instance_name ]

        # first identify all parameters declared in the code 
        code = "\\n".join( code_template_list )
        params = re.findall( r"\\{#[^#]+#\\}", code )
        params = { k: True for k in params }
        params = [ *params ]

        # retrieve parameter value
        for param in params:
            param = re.sub( r"[\\{\\}#]", "", param )
            param = param.replace( " ", "" )

            # we can write template|1 (or any other complement after the pipe ) 
            # to use "template" as param but to force
            # different regular expression replacement if the same parameter
            # appear several times but have to be replaced in different ways 
            p_inst = param

            if "|" in param:
                param = param[ 0: param.find( "|" )]

            # "solid" is the solid template 
            # "instance" is the solid instance

            # precision is a special variable
            if param == "precision":
                param = str( Surface.precision())

            if "solid." == param[ 0:6 ]:
                tokens = param[ 6: ].split( "." )
                param = templ

                for i in range( 0, len( tokens )):
                    param = param[ tokens[ i ]]

            param = str( param )

            if "instance." == param[ 0:9 ]:
                tokens = param[ 9: ].split( "." )
                param = inst

                # note that this will generate an error if inst is None
                for i in range( 0, len( tokens )):
                    param = param[ tokens[ i ]]

            param = str( param )

            if not extra_context is None:
                if param[ 0:( param.find( "." ) + 1 )] == ( ctxt_name + "." ):
                    tokens = param[( param.find( "." ) + 1 ): ].split( "." )
                    param = extra_context

                    for i in range( 0, len( tokens )):
                        param = param[ tokens[ i ]]

            param = str( param )

            # replace param value if necessary
            if not params_dict is None:
                if p_inst in params_dict:
                    rexp = params_dict[ p_inst ][ "rexp" ]
                    repl = params_dict[ p_inst ][ "repl" ]
                    param = re.sub( rexp, repl, param )

            # replace in the code
            code = code.replace( "{# " + p_inst + " #}", param )

        # now check indirections
        indirs = re.findall( r"\\{=>[^#]+#\\}", code )
        indirs = { k: True for k in indirs }
        indirs = [ *indirs ]

        # retrieve indirection command
        grammar = _this.grammar[ _this.syntax ]
        if "curves" in grammar:
            for cmd in indirs:
                _cmd = re.sub( r"[\\{\\}#]", "", cmd[ 3: ] )
                _cmd = _cmd.replace( " ", "" )

                # if cmd is "sketch", look for geometry in grammar
                if _cmd[ 0:7 ] == "sketch.":
                    _cmd = _cmd[ 7: ]

                    # find the sketch
                    sketch = model[ "sketch" ][ _cmd ]

                    if sketch[ "geom" ][ "type" ] in grammar[ "curves" ]:
                        subgrammar = grammar[ "curves" ][ sketch[ "geom" ][ "type" ]]
                        subpars = None if not "params" in subgrammar else subgrammar[ "params" ]
                        subcode = _this._generate_code( template_name, instance_name, subgrammar[ "code" ], subpars, sketch, "sketch" )

                        # replace the indirection
                        code = code.replace( cmd, subcode )

        # finally, {# name {# ... # }} will have been replaced to {# name xxxx #} 
        # and is used to assure valid names for the syntax

        if "{# name" in code:
            rexp = None
            if "valid-names" in _this.grammar[ _this.syntax ]:
                rexp = _this.grammar[ _this.syntax ][ "valid-names" ]

            # extract all names
            names = re.findall( r"\\{# name [^#]+#\\}", code )
            names = { k: True for k in names }
            names = [ *names ]

            for name in names:
                _name = name[ 8: -2 ]

                if not rexp is None:
                    _name = re.sub( rexp[ "rexp" ], rexp[ "repl" ], _name ) 

                code = code.replace( name, _name )

        return code


    @staticmethod
    def template_definitions( strio, template_name ):
        _this = _exporter
        grammar = _this.grammar[ _this.syntax ]

        # check the method that is been used to generate the geometry
        templ = model[ "solid" ][ template_name ]

        code = ""

        if templ[ "cmd" ] == "extrude":
            if "capabilities" in grammar:
                _output = False

                if "extrusion" in grammar[ "capabilities" ]:
                    grammar = _this.grammar[ _this.syntax ][ "capabilities" ][ "extrusion" ]
                    _output = True

                else:
                    if "instance-definitions" in grammar[ "capabilities" ]:
                        grammar = _this.grammar[ _this.syntax ][ "capabilities" ][ "high-level-geometry" ]
                        _output = True

                if _output:
                    code = _this._generate_code( template_name, "", grammar[ "code" ], grammar[ "params" ])

        if code != "":
            _this.flush( strio )
            _this.write( strio, code )
            _this.flush( strio )


    @staticmethod
    def instance_definitions( strio, template_name, instance_name ):
        _this = _exporter
        grammar = _this.grammar[ _this.syntax ]

        code = ""

        pars = None if "params" not in grammar[ "instance" ] else grammar[ "instance" ][ "params" ]
        code = _this._generate_code( template_name, instance_name, grammar[ "instance" ][ "code" ], pars )

        # transforms: translation and rotation
        instance = model[ "instance" ][ instance_name ]

        # rotation first
        if "rot" in instance:
            if instance[ "rot" ] != (0., 0., 0. ):
                rot = grammar[ "rotation" ][ "code" ]

                pars = None if not "params" in grammar[ "rotation" ] else grammar[ "rotation" ][ "params" ]
                transf = _this._generate_code( template_name, instance_name, rot, pars )

                # openscad implements a declarative language. In this case, transforms
                # preceeds object declaration. In all other cases, transforms have to
                # be made after obj creation (as far as I know)

                if "declarative" not in grammar:
                    code = code + chr( 10 ) + transf + chr( 10 )
                else:
                    code = transf + chr( 10 ) + code + chr( 10 )

        # translation afterwards
        if "pos" in instance:
            if instance[ "pos" ] != (0., 0., 0. ):
                move = grammar[ "translation" ][ "code" ]

                pars = None if not "params" in grammar[ "translation" ] else grammar[ "translation" ][ "params" ]
                transf = _this._generate_code( template_name, instance_name, move, pars )

                if "declarative" not in grammar:
                    code = code + chr( 10 ) + transf + chr( 10 )
                else:
                    code = transf + chr( 10 ) + code + chr( 10 )

        if code != "":
            _this.flush( strio )
            _this.write( strio, code )
            _this.flush( strio )



class Implement:
    class _mode:
        development = False

        def __init__( self ):
            pass

        def dev( self ):
            self.development = True

        def prod( self ):
            self.development = False

    mode = _mode()

    def __init__( self ):
        pass

    @staticmethod
    def is_well_formed():
        msgs = []
        res = True

        if "instance" not in model:
            msgs.append( 'No instances of objects. Use Solid.intance( id, model, pos )')
            res = False

        templ_err = ""

        if "solid" not in model:
            msgs.append( "No object templates. Use one of the Solid methods to create a template." )
            res = False

        return res, msgs


    def extrude( template_name ):
        # if the output format has extrusion capability we don't need to define
        # other faces
        extrude = True
        grammar = _exporter.grammar[ _exporter.syntax ]
        if "capabilities" in grammar:
            if "extrusion" in grammar[ "capabilities" ]:
                extrude = False

        templ = model[ "solid" ][ template_name ]

        # extrude implies a sketch
        _sketch_id = templ[ "sketch" ]
        sketch = model[ "sketch" ][ _sketch_id ]

        # generate a face with the sketch
        res = { "faces": [ Surface.face( _sketch_id )]}
        res[ "faces" ][ 0 ][ "vx_offset" ] = 0

        # duplicate the face, if the syntax needs it

        if extrude:
            up = (0.,0.,1.) # sketch[ "up" ]
            _len = templ[ "length" ]
            nface = Surface.move( Surface.face( _sketch_id ), direction=up, length=_len )
            nface[ "vx_offset" ] = len( res[ "faces" ][ 0 ][ "v" ])

            res[ "faces" ].append( nface )

            # create a connection (conns are wires in two faces that have
            # to be tesselated - if we have just the face idxs and no 
            # wires, we have to connect all the vertices of both faces)
            res[ "conns" ] = [{ "faces": [ 0, 1 ], "wires0": [], "wires1": []}]

        return res


    @staticmethod
    def blender():
        global model

        # translate instance of solids to Blender Python
        bpy = StringIO()
        bpy.write( "import bpy" + chr( 10 ));

        well_formed, msgs = Implement.is_well_formed()

        if well_formed == False:
            for msg in msgs:
                bpy.write( "print( '" + msg + "' )" + chr( 10 ))

        if well_formed:
            templates = {}

            # for each instance of a solid
            for _obj in model[ "instance" ]:
                obj = model[ "instance" ][ _obj ]

                # must have a template
                if "template" not in obj:
                    bpy.write( "print( 'ERROR: instance " + _obj + " without template.')" + chr( 10 ))
                else:
                    # if this template have not been processed yet
                    _templ = obj[ "template" ] 
                    if _templ not in templates:

                        # template (in the model, not in this code) must exist
                        if _templ not in model[ "solid" ]:
                            templ_err = "Use one of the Solid methods to create a template."
                            bpy.write( "print( 'ERROR: template " + _templ + " not found. " + templ_err + "')" + chr( 10 ))

                        else:
                            # check the method that is been used to generate the template
                            templ = model[ "solid" ][ _templ ]
                            if templ[ "cmd" ] == "extrude":
                                templates[ _templ ] = Implement.extrude( _templ )

                    # generate blender commands
                    bpy.write( "# python list to hold Blender vertices" + chr( 10 ))
                    bpy.write( "pyvxs = [" )

                    vx_offset = 0

                    for face in templates[ _templ ][ "faces" ]:
                        for vx in face[ "v" ]:
                            bpy.write( str( vx ) + ", " + chr( 10 ))

                    bpy.write( "]" + chr( 10 ))

                    bpy.write( "# python list to hold vertex indices that define faces" + chr( 10 ))
                    bpy.write( "faces = [" + chr( 10 ))

                    for face in templates[ _templ ][ "faces" ]:

                        # if there is no edges, a natural order is implied
                        if "f" not in face:
                            _f = list( range( int( face[ "vx_offset" ]), int( face[ "vx_offset" ]) + len( face[ "v" ])))
                            _f.append( int( face[ "vx_offset" ]))
                            _f = str( tuple( _f ))
                        else:
                            _f = face[ "f" ]

                        _f = _f.replace( "),", ")," + chr( 10 )) + ", " + chr( 10 )
                        bpy.write( _f + chr( 10 ))

                    bpy.write( "]" + chr( 10 ))

                    bpy.write( 'mesh_data = bpy.data.meshes.new( "' + _obj + '_data" )' + chr( 10 ))
                    bpy.write( "mesh_data.from_pydata( pyvxs, [], faces )" + chr( 10 ))
                    bpy.write( "mesh_data.update()" + chr( 10 ))

                    bpy.write( 'obj = bpy.data.objects.new( "' + _obj + '", mesh_data )' + chr( 10 ))

                    bpy.write( "scene = bpy.context.scene" + chr( 10 ))
                    bpy.write( "scene.collection.objects.link( obj )" + chr( 10 ))

        res = bpy.getvalue()
        bpy.close()
        return res

    @staticmethod
    def wavefront():
        global model

        # translate instance of solids to alias wavefront obj format
        obj = StringIO()
        obj.write( "# clip-bolt jupyter CAD/CAM" +  chr( 10 ));

        if assert_model([ "asset", "id" ]):
            id = model[ "asset" ][ "id" ]
            obj.write( "# modelo " + model[ "asset" ][ "id" ] + chr( 10 ));
            obj.write( "o " + model[ "asset" ][ "id" ] + chr( 10 ));

        if assert_model([ "asset", "description" ]):
            desc = model[ "asset" ][ "description" ]
            obj.write( "# " + model[ "asset" ][ "description" ] +  chr( 10 ));

        well_formed, msgs = Implement.is_well_formed()

        if well_formed == False:
            for msg in msgs:
                obj.write( "# " + msg + chr( 10 ))

        if well_formed:
            templates = {}

            # for each instance of a solid
            for _solid in model[ "instance" ]:
                solid = model[ "instance" ][ _solid ]

                # must have a template
                if "template" not in solid:
                    obj.write( "# ERROR: instance " + _solid + " without template" + chr( 10 ))
                else:
                    # if this template have not been processed yet
                    _templ = solid[ "template" ] 
                    if _templ not in templates:

                        # template (in the model, not in this code) must exist
                        if _templ not in model[ "solid" ]:
                            templ_err = "Use one of the Solid methods to create a template."
                            obj.write( "# ERROR: template " + _templ + " not found. " + templ_err + chr( 10 ))

                        else:
                            # check the method that is been used to generate the template
                            templ = model[ "solid" ][ _templ ]
                            if templ[ "cmd" ] == "extrude":
                                templates[ _templ ] = Implement.extrude( _templ )

                    # generate obj file
                    obj.write( "# vertices" + chr( 10 ))

                    vx_offset = 0

                    for face in templates[ _templ ][ "faces" ]:
                        for vx in face[ "v" ]:
                            obj.write( "v " + str( vx ).replace( ",", " " ).replace( "(", "" ).replace( ")", "" ) + chr( 10 ))

                    # vertex normals
                    for face in templates[ _templ ][ "faces" ]:
                        for vx in face[ "vn" ]:
                            obj.write( "vn " + str( vx ).replace( ",", " " ).replace( "(", "" ).replace( ")", "" ) + chr( 10 ))

                    obj.write( "# faces" + chr( 10 ))

                    for face in templates[ _templ ][ "faces" ]:
                        # if there is no edges, a natural order is implied
                        if "f" not in face:
                            _f = list( range( int( face[ "vx_offset" ]), int( face[ "vx_offset" ]) + len( face[ "v" ])))
                            _f.append( int( face[ "vx_offset" ]))
                            _f = str( tuple( _f ))
                        else:
                            _f = face[ "f" ]

                        _f = _f.replace( "(", "" ).replace( ")", "" ).replace( ",", " " )
                        _f = re.sub( r"(\d+)", r"\\1//\\1", _f )
                        obj.write( "f " + _f + chr( 10 ))

        res = obj.getvalue()
        obj.close()
        return res


    @staticmethod
    def _version( syntax ):
        global model
        _e = _exporter

        # translate instance of solids to a specific syntax
        code = StringIO()
        _e.syntax = syntax
        _e.write( code, _e.comment( "clip-bolt jupyter CAD/CAM" ));

        if assert_model([ "asset", "id" ]):
            id = model[ "asset" ][ "id" ]
            _e.write( code, _e.comment( "modelo " + model[ "asset" ][ "id" ]));

        if assert_model([ "asset", "description" ]):
            desc = model[ "asset" ][ "description" ]
            _e.write( code, _e.comment( model[ "asset" ][ "description" ]));

        well_formed, msgs = Implement.is_well_formed()

        if well_formed == False:
            for msg in msgs:
                _e.write( code, _e.comment( msg ))

        if well_formed:

            _e.initialize( code )

            designs = {}

            # for each instance of a solid
            for _instance in model[ "instance" ]:
                instance = model[ "instance" ][ _instance ]

                # must have a template
                if "template" not in instance:
                    _e.write( code, _e.comment( "ERROR: instance " + _instance + " without template" ))
                else:
                    # create designs ( templates "materialized" for use in this code )
                    _templ = instance[ "template" ] 
                    if _templ not in designs:

                        # template (in the model, not in this code) must exist
                        if _templ not in model[ "solid" ]:
                            templ_err = "Use one of the Solid methods to create a template."
                            _e.write( code, _e.comment( "ERROR: template " + _templ + " not found. " + templ_err ))

                        else:
                            # check the method that is been used to generate the template
                            templ = model[ "solid" ][ _templ ]
                            if templ[ "cmd" ] == "extrude":

                                # some formats are high level and don't need vx and face data, but we generate them
                                # here anyway to be able to deal with implementation specifics of each case

                                # TODO: implementar instancias de solidos, estou testando com uma unica instancia
                                designs[ _templ ] = Implement.extrude( _templ )
                                designs[ _templ ][ "template" ] = templ 

                        # generate output file
                        #   formats with low level geometry definition (they need vx and face data)
                        if _e.verbose():
                            _e.output_vertices( code, _templ, designs[ _templ ])
                            _e.output_faces( code, _templ, designs[ _templ ])

                        #  template definitions
                        # ( if we have seen it before, output just the high level geometry )
                        _e.template_definitions( code, _templ )

                    #  instance definitions
                    # ( if we have seen it before, output just the high level geometry )
                    _e.instance_definitions( code, _templ, _instance )

                    # desing already created, generating another instance of a known design
                    # TODO: else:

        _e.finalize( code )

        res = code.getvalue()
        code.close()
        return res


    @staticmethod
    def vpython():
        return Implement._version( "vpython" )


    @staticmethod
    def openscad():
        return Implement._version( "openscad" )
"""
